Import historical data of **Silver Prices during 2020-2023** using Yahoo finance.

In [1]:
!pip install yfinance

In [2]:
import yfinance as yf

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
# Silver Prices
ticker_symbol="SI=F"
start_date="2020-01-01"
end_date="2024-01-01"

df=yf.download(ticker_symbol,start=start_date,end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,17.966000,17.990000,17.966000,17.966000,17.966000,2
2020-01-03,18.110001,18.110001,17.965000,18.068001,18.068001,83
2020-01-06,18.025000,18.105000,18.025000,18.097000,18.097000,3
2020-01-07,18.014999,18.344999,18.014999,18.316000,18.316000,33
2020-01-08,18.400000,18.504999,18.070000,18.087999,18.087999,31


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2020-01-02 to 2023-12-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1006 non-null   float64
 1   High       1006 non-null   float64
 2   Low        1006 non-null   float64
 3   Close      1006 non-null   float64
 4   Adj Close  1006 non-null   float64
 5   Volume     1006 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 55.0 KB


In [6]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1006.000000,1006.000000,1006.000000,1006.000000,1006.000000,1006.000000
mean,22.745953,22.913692,22.566426,22.732278,22.732278,1506.219682
std,3.205626,3.223612,3.182028,3.203664,3.203664,9202.326533
min,12.070000,12.205000,11.735000,11.735000,11.735000,0.000000
25%,21.005000,21.247499,20.913750,21.051001,21.051001,11.250000
50%,23.347500,23.500000,23.208000,23.345500,23.345500,47.000000
75%,24.950500,25.157500,24.779251,24.922750,24.922750,153.750000
max,29.530001,30.160000,28.313999,29.398001,29.398001,131415.000000


In [7]:
import numpy as np
import pandas as pd

First scale the data.

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))


Create a time series dataset where X encompasses the features and y is our target.

In [10]:
def create_dataset(dataset, time_step=1):
    X, y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 100
X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0],X.shape[1], 1) # Reshape for LSTM [samples, time steps, features]


Split into training and test set.

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Now we can build the LSTM model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [14]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(100, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')


Train/fit the model

In [16]:
from tensorflow.keras.callbacks import  EarlyStopping
#So that overfitting can be avoided

In [17]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=10)])


Epoch 1/100
12/12 [==============================] - 7s 191ms/step - loss: 0.1325 - val_loss: 0.0059
Epoch 2/100
12/12 [==============================] - 1s 114ms/step - loss: 0.0149 - val_loss: 0.0055
Epoch 3/100
12/12 [==============================] - 2s 195ms/step - loss: 0.0073 - val_loss: 0.0050
Epoch 4/100
12/12 [==============================] - 1s 118ms/step - loss: 0.0057 - val_loss: 0.0049
Epoch 5/100
12/12 [==============================] - 1s 112ms/step - loss: 0.0050 - val_loss: 0.0046
Epoch 6/100
12/12 [==============================] - 1s 111ms/step - loss: 0.0047 - val_loss: 0.0048
Epoch 7/100
12/12 [==============================] - 1s 112ms/step - loss: 0.0044 - val_loss: 0.0043
Epoch 8/100
12/12 [==============================] - 1s 111ms/step - loss: 0.0041 - val_loss: 0.0042
Epoch 9/100
12/12 [==============================] - 1s 108ms/step - loss: 0.0040 - val_loss: 0.0040
Epoch 10/100
12/12 [==============================] - 1s 110ms/step - loss: 0.0038 - val_lo

Make predictions.

In [18]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Transform back to original form
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)


6/6 [==============================] - 0s 24ms/step


Evaluate using root mean squared error.

In [19]:
from sklearn.metrics import mean_squared_error
# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, scaler.inverse_transform(np.array(y_train).reshape(-1,1))))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predict))
print(f'Train RMSE: {train_rmse}, Test RMSE: {test_rmse}')


Train RMSE: 22.898066454605072, Test RMSE: 22.880682967144924


This is a very high error and is a very undesirable outcome. This model with these specific parameters is not suitable for our data unfortunately.